# 3. CAPM

<p style="font-size:28px;">1. Summary Statistics</p>

In [50]:
import numpy as np
import pandas as pd
import statistics
data=pd.read_excel('dfa_analysis_data.xlsx', sheet_name='portfolios (total returns)')
START_YR=1981
df_subsample = data[(data['Date'].dt.year>= START_YR)]
df_subsample.drop(['Date'],axis=1, inplace=True)
df_subsample.dropna(inplace=True)


C:\Users\ABC\AppData\Local\Temp\ipykernel_18352\2773500845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subsample.drop(['Date'],axis=1, inplace=True)
C:\Users\ABC\AppData\Local\Temp\ipykernel_18352\2773500845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subsample.dropna(inplace=True)


In [51]:
data2=pd.read_excel('dfa_analysis_data.xlsx', sheet_name='factors')
data2 = data2[(data2['Date'].dt.year>= START_YR)]
df_rf=data2['RF']


In [52]:
df_excess_ret = df_subsample.sub(df_rf, axis=0)
Scale_factor=12
annualizedMean=df_excess_ret.mean()*Scale_factor        #Annualized Mean
annualizedVol=df_excess_ret.std()*np.sqrt(Scale_factor) #Annualized Volatility
sharpe=annualizedMean/annualizedVol
newData=pd.concat([annualizedMean,annualizedVol,sharpe ],axis=1)
newData.columns=['Annualized Mean', 'Annualized Volatility', 'Sharpe Ratio',]
dataSorted=df_excess_ret.sort_values(by=list(df_excess_ret.columns), na_position="first")
var=np.quantile(dataSorted, 0.05,axis=0).T
newData['Var(0.05)']=var
newData


,Annualized Mean,Annualized Volatility,Sharpe Ratio,Var(0.05)
SMALL LoBM,0.011705,0.271737,0.043075,-0.124886
ME1 BM2,0.088438,0.235431,0.375641,-0.094929
ME1 BM3,0.090226,0.200834,0.449260,-0.084817
ME1 BM4,0.112535,0.194031,0.579986,-0.077604
SMALL HiBM,0.127314,0.208354,0.611049,-0.088174
ME2 BM1,0.060922,0.244669,0.248999,-0.103232
ME2 BM2,0.098365,0.205371,0.478962,-0.083392
ME2 BM3,0.105155,0.186431,0.564042,-0.080313
ME2 BM4,0.108069,0.181868,0.594217,-0.075338
ME2 BM5,0.113192,0.213654,0.529793,-0.093275



<p style="font-size:28px;">2. CAPM</p>

In [53]:
import statsmodels.api as sm


betas = pd.Series(np.nan, index=df_excess_ret.columns)
alphas = pd.Series(np.nan, index=df_excess_ret.columns)
df_residuals = pd.DataFrame(columns=df_excess_ret.columns)

summed = 0
x = sm.add_constant(data2['Mkt-RF'])  

for col in df_excess_ret.columns:

    y = df_excess_ret[col]
    model = sm.OLS(y, x).fit()
    alpha, beta = model.params["const"], model.params['Mkt-RF']
    df_residuals[col] =df_excess_ret[col] -alpha -beta*(data2['Mkt-RF'])
    betas.loc[col] = beta
    alphas.loc[col] = alpha

for i in alphas:
    summed=abs(i)+summed
treynor =annualizedMean/betas


ann_tracking_err = np.sqrt(Scale_factor) * df_residuals.std()
annualized_alphas = Scale_factor * alphas
info_ratio = annualized_alphas / ann_tracking_err

df_CAPM = pd.DataFrame({"Alpha(Annualized)":Scale_factor * alphas,"Beta": betas,'Treynor':treynor,"Information Ratio":info_ratio})
print(f"Mean Absolute Error: {summed/len(df_excess_ret.columns):.6f}")
df_CAPM


Mean Absolute Error: 0.001721


,Alpha(Annualized),Beta,Treynor,Information Ratio
SMALL LoBM,-0.103740,1.358486,0.008616,-0.604694
ME1 BM2,-0.010629,1.165759,0.075863,-0.070510
ME1 BM3,0.001039,1.049509,0.085970,0.008831
ME1 BM4,0.029481,0.977337,0.115145,0.243461
SMALL HiBM,0.042851,0.993918,0.128093,0.305768
ME2 BM1,-0.052447,1.334065,0.045667,-0.401824
ME2 BM2,0.001576,1.138954,0.086364,0.015050
ME2 BM3,0.017142,1.035676,0.101532,0.181248
ME2 BM4,0.025089,0.976453,0.110675,0.249250
ME2 BM5,0.018794,1.110819,0.101900,0.148788


If the CAPM were true, 
1) We would expect the MAE to be very close to 0, because the model would perfectly explain expected returns and all pricing errors would vanish.
2) All assets and portfolios should have the same Treynor Ratio (equal to the market risk premium).
3) All assets should have alpha = 0, therefore, all assets should have Information Ratio = 0.

<p style="font-size:28px;">3. Cross-sectional Estimation</p>

In [54]:

df_crossCAPM = pd.concat([annualizedMean.rename('Excess Return'),betas.rename('Beta')], axis=1)
X1 = sm.add_constant(df_crossCAPM['Beta'])

y1 = df_crossCAPM['Excess Return']

model = sm.OLS(y1, X1).fit()

r_squared = model.rsquared
intercept = model.params['const']
slope = model.params['Beta']

print(f"R-squared: {r_squared:.4f}")
print(f"Intercept: {intercept:.6f}")
print(f"Market Risk Premium: {slope:.6f}")

R-squared: 0.3132
Intercept: 0.205842
Market Risk Premium: -0.105912


Every asset’s mean excess return lies exactly on the Security Market Line, so the regression fits perfectly with no pricing errors.</br>
R-squared: 1</br>
Intercept: 0</br>
Market Risk Premium: Expected market risk premium

<p style="font-size:28px;">4. Conclusion</p>

If CAPM were true, all alphas =0, and R Sq =1. Since they are not, DFA’s multi-factor philosophy is validated.